#Idea
idea of this fusion model: our 2 model gives a vector of 27 element as output,
we want to freeze both pretrained model, and just combine the output to classify better than on by one

il faudrait qu'il y ai une couche qui fasse : $a*out_{image}+b*out_{text}$
avec a et b des params entraînables

# Hyperparameters

In [150]:
BATCH_SIZE=275#10.4/15GO
num_classes=27

train_percentage=0.80
test_percentage=0.10
valid_percentage=0.10


# Imports


In [151]:
!pip install unidecode

In [152]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [153]:

import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader
from torch.optim import lr_scheduler
import torchvision

import imageio

from tqdm.auto import tqdm # for cool loading bar


# import for NLP
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from keras.utils import to_categorical

from unidecode import unidecode
from nltk.corpus import stopwords



from sklearn.metrics import f1_score

import zipfile
import os
import copy

import matplotlib.pyplot as plt


In [154]:
# mount the drive where your dataset is availabledevice = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
from google.colab import drive
drive.mount('/content/drive')
filepath='/content/drive/MyDrive/datasets/multimodal_product_classification/' # add your own path. Where to save the dataset

if not os.path.exists('datasets'):
  os.makedirs('datasets')
  with zipfile.ZipFile(filepath+'images.zip', 'r') as zip_ref:
      zip_ref.extractall('datasets')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [155]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


# Preprocessing

In [156]:
# Load data
X_train = pd.read_csv(filepath+'X_train.csv')
y_train = pd.read_csv(filepath+'Y_train.csv')
X_train=X_train.drop(columns="Unnamed: 0")
y_train=y_train.drop(columns="Unnamed: 0")


In [157]:
# Cleaning and Preprocessing Text
CLEANR = re.compile('<.*?>') # delete html tag

def clean_html(raw_html):
  cleantext = re.sub(CLEANR, '', raw_html)
  return cleantext
def clean_special(text):
    text = re.sub(r'[^a-z% ]', '', text)
    return(text)

def clean_text(text):
    # Convert text to lowercase
    text = text.lower()
    text=re.sub('\d', '%', text)#replace all digits by %
    text=re.sub("'", ' ', text)#replace all ' by a space

    text = unidecode(text)
    text=clean_html(text)
    text=clean_special(text)
    #clean stopwords
    text_list=text.split()
    text_list=[word for word in text_list if word not in final_stopwords_list]
    text=" ".join(text_list)
    return(text)

final_stopwords_list = stopwords.words('english') + stopwords.words('french')+stopwords.words('german')# Fr,En,De Stopwords


# Apply cleaning function to the 'designation' column
X_train['designation'] = X_train['designation'].fillna('').apply(clean_text)
X_train['description'] = X_train['description'].fillna('').apply(clean_text)


In [158]:
pd.set_option('display.max_colwidth', 150)
X_train.designation.head(5)


0                                                                           olivia personalisiertes notizbuch %%% seiten punktraster ca din a% rosendesign
1    journal arts ndeg %%% %%%%%%%% art marche salon art asiatique paris jacques barrere francois perrier reforme ventes encheres publiques sna fete ce...
2                                                                               grand stylet ergonomique bleu gamepad nintendo wii u speedlink pilot style
3                                                                                                  peluche donald europe disneyland %%%% marionnette doigt
4                                                                                                                                            guerre tuques
Name: designation, dtype: object

In [159]:
X_train.description.head(5)


0                                                                                                                                                         
1                                                                                                                                                         
2    pilot style touch pen marque speedlink % stylet ergonomique gamepad nintendo wii u confort optimal precision maximale gamepad wii u grand stylet h...
3                                                                                                                                                         
4          luc ideacutees grandeur veut organiser jeu guerre boules neige arranger ecirctre vainqueur incontesteacute sophie mecircle chambarde tous plans
Name: description, dtype: object

In [160]:
# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train['designation'])
designation_sequences = tokenizer.texts_to_sequences(X_train['designation'])
description_sequences = tokenizer.texts_to_sequences(X_train['description'])


In [161]:
designation_maxlen=max(len(x) for x in designation_sequences)#maxlen for designation
designation_maxlen

40

In [162]:
max(len(x) for x in tokenizer.texts_to_sequences(X_train['description']))#max description size is 1090


1090

In [163]:
import plotly.express as px
desc_len=[len(x) for x in description_sequences]
fig = px.histogram(pd.DataFrame(desc_len,columns=["x"]), x="x",
                   labels={
                     "x": "description Length (number of tokens)",
                 })
fig.show()


200 tokens seems to be a good value to cut to, we see that about 50% of the product don't have description

In [164]:
# Padding to max length of text
padded_designation=pad_sequences(designation_sequences,maxlen=designation_maxlen)
padded_description=pad_sequences(description_sequences,maxlen=200)

In [165]:
for i, row in X_train.iterrows():
    X_train.at[i, 'designation'] = padded_designation[i]
    X_train.at[i, "description"] = padded_description[i]

In [166]:
X_train

,designation,description,productid,imageid
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8547, 28594, 28595, 28596, 28597, 2711, 4136, 30, 28...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",3804725264,1263597046
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 39, 891, 3, 130, 777, 314, 130, 11574, 163, 1375, 15867, 1920, 15868, 4446, 5510, 7892, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",436067568,1008141237
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 182, 4137, 1438, 33, 1028, 114, 267, 193, 13314, 1156, 226]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",201115110,938777978
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 101, 2887, 836, 9339, 2841, 3029]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",50418756,457047496
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 365, 28600]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",278535884,1077757786
...,...,...,...,...
84911,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2644, 118, 568]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",206719094,941495734
84912,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 4, 43, 8836, 564, 264, 18, 1, 18]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",3065095706,1188462883
84913,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 39, 1974, 1425, 371, 3, 1727, 8509, 779, 2173, 779, 446, 2173, 2527, 2015, 1971, 19511, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",440707564,1009325617
84914,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 59, 666, 15, 5017, 591, 502, 449, 1, 1, 2]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",3942400296,1267353403


In [167]:
# Assuming the number of unique words in the tokenizer plus 1 is vocab_size
vocab_size = len(tokenizer.word_index) + 1
print(f'{vocab_size=}')# nearly 64517 of vocab size, it seems too much :/

vocab_size=64517


In [168]:
y_train=y_train["prdtypecode"].to_numpy()
# Convert labels to categorical
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
# Convert to one-hot encoding
y_train_categorical = to_categorical(y_train_encoded)


In [169]:
y_train=pd.DataFrame(y_train,columns=["labels"])

In [170]:
train_df=pd.concat([X_train,y_train],axis=1)

In [171]:
train_df

,designation,description,productid,imageid,labels
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8547, 28594, 28595, 28596, 28597, 2711, 4136, 30, 28...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",3804725264,1263597046,10
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 39, 891, 3, 130, 777, 314, 130, 11574, 163, 1375, 15867, 1920, 15868, 4446, 5510, 7892, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",436067568,1008141237,2280
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 182, 4137, 1438, 33, 1028, 114, 267, 193, 13314, 1156, 226]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",201115110,938777978,50
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 101, 2887, 836, 9339, 2841, 3029]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",50418756,457047496,1280
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 365, 28600]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",278535884,1077757786,2705
...,...,...,...,...,...
84911,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2644, 118, 568]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",206719094,941495734,40
84912,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 4, 43, 8836, 564, 264, 18, 1, 18]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",3065095706,1188462883,2583
84913,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 39, 1974, 1425, 371, 3, 1727, 8509, 779, 2173, 779, 446, 2173, 2527, 2015, 1971, 19511, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",440707564,1009325617,2280
84914,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 59, 666, 15, 5017, 591, 502, 449, 1, 1, 2]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",3942400296,1267353403,1560



# Data loading

In [172]:
def image_to_tensor(image):
    img=torch.from_numpy(image).float()
    out=img.permute(2,0,1)
    return(out)

In [235]:
class ImageTextDataLoader(Dataset):
    """Title, Description and Image dataset."""

    def __init__(self, dataframe, image_dir, transform=None):
        """
        Arguments:
            csv_file (string): Path to the csv file with annotations.
            image_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.df = dataframe
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        selected_df=self.df.iloc[idx]
        print(selected_df["imageid"])


        image_name="image_"+str(selected_df["imageid"])+"_product_"+str(selected_df["productid"])+".jpg"
        filepath=os.path.join(self.image_dir,image_name)
        image_arr = imageio.v3.imread(filepath)
        t_image=image_to_tensor(image_arr)

        if self.transform:
           t_image = self.transform(t_image)

        print(selected_df['designation'])
        print(selected_df['description'])
        print(selected_df['labels'])

        designation=selected_df['designation']
        description=selected_df['description']
        label=selected_df['labels']


        t_designation=torch.tensor(designation).to(device)
        t_description=torch.tensor(description).to(device)
        t_label=torch.tensor(label).to(device)

        return [t_designation,t_description,t_image,t_label]


In [236]:
transform = torchvision.transforms.Compose([
            torchvision.transforms.Resize(256,antialias=True),
            torchvision.transforms.CenterCrop(224),
            torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
       ])

In [237]:
dataset=ImageTextDataLoader(train_df,"/content/datasets/images/image_train",transform)


In [238]:
train_size = int(train_percentage * len(dataset))
valid_size = int(test_percentage * len(dataset))
test_size = len(dataset) - train_size - valid_size

train_dataset, valid_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, valid_size, test_size])
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)


dataloaders={
    'train':train_dataloader,
    'val':valid_dataloader,
    'test':test_dataloader
    }


# Vision Model Loading

In [239]:
model = torchvision.models.resnet18(weights="IMAGENET1K_V1")

# Replace the last fully-connected layer
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, num_classes) # num_classes is the number of classes in your dataset

model.load_state_dict(torch.load("/content/drive/MyDrive/Lessons/Models/multimodal_classification/Resnet16-best10.ckpt",map_location=device))


<All keys matched successfully>